**2.4   Mutable Data**

***2.4.1   The Object Metaphor***

In [1]:
from datetime import date

In [4]:
print(date(2024,12,16))
tues = date(2024,12,7)

2024-12-16


In [5]:
print(date(2024, 12, 16) - date(2024, 12, 15))

1 day, 0:00:00


In [8]:
tues.strftime('%A, %B %d')

'Saturday, December 07'

In [9]:
'123'.isnumeric()

True

In [10]:
'rOBERT dE nIRO'.swapcase()

'Robert De Niro'

In [11]:
a =5
b = a
a is b

True

In [12]:
b = 4
a

5

In [13]:
chinese = ['coin', 'string', 'myriad']
suits = chinese
suits is chinese

True

In [14]:
suits.pop()

'myriad'

In [15]:
chinese

['coin', 'string']

In [17]:
suits.append('cup')
suits.extend(['sword', 'club'])

In [18]:
chinese

['coin', 'string', 'cup', 'cup', 'sword', 'club']

In [19]:
chinese is suits

True

In [20]:
suits[2] = 'spade'
chinese

['coin', 'string', 'spade', 'cup', 'sword', 'club']

In [21]:
nest = list(suits)
nest is suits

False

In [23]:
nest[0] = suits
nest

[['coin', 'string', 'spade', 'cup', 'sword', 'club'],
 'string',
 'spade',
 'cup',
 'sword',
 'club']

In [24]:
suits.insert(2, 'Joker')
nest

[['coin', 'string', 'Joker', 'spade', 'cup', 'sword', 'club'],
 'string',
 'spade',
 'cup',
 'sword',
 'club']

In [25]:
nest[0].pop(2)

'Joker'

In [26]:
suits

['coin', 'string', 'spade', 'cup', 'sword', 'club']

In [27]:
suits is nest[0]

True

In [28]:
()

()

In [29]:
(1,)

(1,)

In [30]:
code = ("up", "up", "down", "down") + ("left", "right") * 2
code

('up', 'up', 'down', 'down', 'left', 'right', 'left', 'right')

In [31]:
code.index('left')

4

However, the methods for manipulating the contents of a list are not available for tuples because tuples are immutable.

While it is not possible to change which elements are in a tuple, it is possible to change the value of a mutable element contained within a tuple.

In [32]:
nest = (10, 20, [30, 40])
nest[2].pop()

40

In [33]:
nest

(10, 20, [30])

In [34]:
dict([(3, 9), (4, 16), (5, 25)])

{3: 9, 4: 16, 5: 25}

In [36]:
numerals = {'I': 1, 'V': 5, 'X': 10}

In [38]:
numerals.get('V', 0)

5

In [39]:
{x: x*x for x in range(3,6)}

{3: 9, 4: 16, 5: 25}

In [79]:
def make_withdraw(balance):
    def withdraw(amount):
        nonlocal balance
        if amount > balance:
            return 'Insufficient funds'
        balance -= amount
        return balance
    return withdraw
wd = make_withdraw(100)
wd2 = wd
wd2(20)
wd(20)
wd2 is wd

True

The key to correctly analyzing code with non-local assignment is to remember that ***only function calls can introduce new frames***. Assignment statements always change bindings in existing frames. In this case, unless make_withdraw is called twice, there can be only one binding for balance.

In [76]:
wd2 = make_withdraw(7)
wd2(2)

5

In [74]:
b = 3
c = 1
def f(x):
    global c
    if c == 1:
        return b
    c = c + 1 #
    return c
f(1)

3

This UnboundLocalError appears because **c** is assigned locally in line ***7***, and so Python assumes that all references to c must appear in the local frame as well. 

*2.4.7   Implementing Lists and Dictionaries*

In [80]:
def mutable_link():
    contents = empty
    def dispatch(message, value=None):
        nonlocal contents
        if message == 'len':
            return len_link(contents)
        elif message == 'push_first':
            contents = link(value, contents)
        elif message == 'pop_first':
            f = first(contents)
            contents = rest(contents)
            return f
        elif message == 'str':
            return join_link(contents, ', ')
    return dispatch


In [81]:
def to_mutable_link(source):
    s = mutable_link()
    for element in reversed(source):
        s('push_first', element)
    return s

Implementing Dictionaries. We can also implement a value with similar behavior to a dictionary. In this case, we use a list of key-value pairs to store the contents of the dictionary. Each pair is a two-element list.

In [84]:
def dictionary():
    records = []
    def getitem(key):
        matches = [r for r in records if r[0] == key]
        if len(matches) ==1:
            key, value = matches[0]
            return value
    def setitem(key, value):
        nonlocal records
        non_matches = [r for r in records if r[0] != key]
        records = non_matches + [[key, value]]
    def dispatch(message, key=None, value=None):
        if message == 'getitem':
            return getitem(key)
        elif message == 'setitem':
            setitem(key, value)
    return dispatch

In [85]:
d = dictionary()
d('setitem', 3, 9)

In [86]:
d('getitem', 3)

9